Code to load environment data from a PiE server and make plots with plotly

Each of these code block need to be run in sequence.

In [10]:
# import all the required libraries

from __future__ import print_function

import time
from datetime import datetime, timedelta

import ipywidgets as widgets
from IPython.display import display

import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go

import urllib2 # python 2
from StringIO import StringIO # python 2

# check plotly version, version >3.0 is needed
#import plotly
#print('plotly version:', plotly.__version__)

Define global variables

In [16]:
gServerUrl = 'http://192.168.1.19:5010/environmentlog'
gNumberOfDays = 7
gEndDate = datetime.now()
gStartDate = gEndDate - timedelta(days=gNumberOfDays)
gMinY = 0 #50
gMaxY = 100

In [17]:
# load all data from PiE server

# todo: set globals gStartDate/gEndDate from first/last entry in df

df = None
def loaddata():
    url = gServerUrl #'http://192.168.1.19:5010/environmentlog'
    response = urllib2.urlopen(url)
    html = response.read() # this is full text of the log file
    html = StringIO(html)
    global df
    df = pd.read_csv(html)

    # construct a plotly friendly date/time, format is 'yyyy-mm-dd HH:MM:SS'
    def tmpFunc(seconds):
        return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(seconds))
                         
    # append a new column to df called 'DateTime'
    df['DateTime'] = df['Seconds'].apply(lambda x: tmpFunc(x))

    # celcius to farenheight, (0°C × 9/5) + 32 = 32°F
    def c2f(c):
        return (c * 9.0/5.0) + 32.0
    df['farenheight'] = df['Temperature'].apply(lambda x: c2f(x))

loaddata()

In [18]:
# plot with plotly

#fig = None
f0 = None
def myplot():
    #print('myplot() gStartDate:', gEndDate)
    
    startSeconds = time.mktime(gStartDate.timetuple())
    stopSeconds = time.mktime(gEndDate.timetuple())
    
    # create a plot dataframe (plot_df) to plot a subset of main dataframe (df)
    plot_df = df[df['Seconds'].between(startSeconds, stopSeconds)] # inclusive

    # extact columns (these are not raw numbers yet)
    myDateTime = plot_df[['DateTime']]
    myTemperature = plot_df[['Temperature']]
    farenheight = plot_df[['farenheight']]
    myHumidity = plot_df[['Humidity']]

    # get the actual numbers from each column
    myDateTime2 = myDateTime.iloc[:]['DateTime']
    myTemperature2 = myTemperature.iloc[:]['Temperature'] * 100 # (0°C × 9/5) + 32 = 32°F
    farenheight2 = farenheight.iloc[:]['farenheight']
    myHumidity2 = myHumidity.iloc[:]['Humidity']


    # set up traces to plot in plotly
    # todo: make either box or vertical lines for daytime/nighttime or for 12AM
    trace1 = go.Scatter(
        x=myDateTime2,
        y=farenheight2,
        name='Temperature',
        mode='markers',
        marker= dict(size= 5,
                    line= dict(width=1), #, color=plot_df['color']),
                    opacity= 1
                    )
    )
    trace2 = go.Scatter(
        x=myDateTime2,
        y=myHumidity2,
        name='Humidity',
        mode='markers',
        yaxis='y2'
    )
    
    #data = [trace1, trace2]
    data = [trace1]
    
    # make a plotly layout
    layout = go.Layout(
        title='Temperature And Humidity',
        yaxis=dict(
            title='Temperature (deg fahrenheit)',
            titlefont=dict(
                # this is same as default plot color #1
                color='#1f77b4'
            ),
            tickfont=dict(
                # this is same as default plot color #1
                color='#1f77b4'
            ),
            range=[gMinY, gMaxY]
        ),
        yaxis2=dict(
            title='Humidity (%)',
            #titlefont=dict(
            #    color='rgb(148, 103, 189)'
            #),
            #tickfont=dict(
            #    color='rgb(148, 103, 189)'
            #),
            titlefont=dict(
                # this is same as default plot color #2
                color='#ff7f0e'
            ),
            tickfont=dict(
                # this is same as default plot color #2
                color='#ff7f0e'
            ),
            overlaying='y',
            side='right'
        )
    )

    # plot with plotly
    #global fig
    fig = go.Figure(data=data, layout=layout)

    global f0
    if f0 is None:
        f0 = go.FigureWidget(fig)
    else:
        #todo: finish sparating plot initialization with myplot() from plot refreshing with myrefresh()
        with f0.batch_update():
            f0.data[0].x = myDateTime2
            f0.data[0].y = farenheight2
            #print('f0:', f0)
            f0['layout']['yaxis']['range'] = [gMinY, gMaxY]

def myrefresh():
    startSeconds = time.mktime(gStartDate.timetuple())
    stopSeconds = time.mktime(gEndDate.timetuple())
    
    # create a plot dataframe (plot_df) to plot a subset of main dataframe (df)
    plot_df = df[df['Seconds'].between(startSeconds, stopSeconds)] # inclusive

    # extact columns (these are not raw numbers yet)
    myDateTime = plot_df[['DateTime']]
    #myTemperature = plot_df[['Temperature']] # celcius
    farenheight = plot_df[['farenheight']]
    #myHumidity = plot_df[['Humidity']]

    # get the actual numbers from each column
    myDateTime2 = myDateTime.iloc[:]['DateTime']
    #myTemperature2 = myTemperature.iloc[:]['Temperature'] # celcius (0°C × 9/5) + 32 = 32°F
    farenheight2 = farenheight.iloc[:]['farenheight']
    #myHumidity2 = myHumidity.iloc[:]['Humidity']

    global f0
    with f0.batch_update():
        f0.data[0].x = myDateTime2
        f0.data[0].y = farenheight2
        #print('f0:', f0)
        f0['layout']['yaxis']['range'] = [gMinY, gMaxY]
    

In [27]:
# ipywidgets interface

#
# number of days (before end date)
style = {'description_width': 'initial', 'width': '20px'}
numberOfDaysWidget = widgets.BoundedIntText(
    value=gNumberOfDays,
    min=0,
    step=1,
    description='Days before end date:',
    disabled=False,
    style=style
)

# callback when user changes number of days
def days_callback(change):
    #print('days_callback():', change['new'])
    numberOfDays = int(change['new'])
    global gStartDate
    gStartDate = gEndDate - timedelta(days=numberOfDays)
    global startDateWidget
    startDateWidget.value = gStartDate    
    #print('gStartDate:', gStartDate)
    myplot()
    
numberOfDaysWidget.observe(days_callback, names='value')

#
# start date
startDateWidget = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
startDateWidget.value = gStartDate

#
# stop date
endDateWidget = widgets.DatePicker(
    description='End Date',
    disabled=False
)
endDateWidget.value = gEndDate


#
# PiE server REST url
urlStyle = {'description_width': 'initial', 'width': '200px'}
serverUrlWidget = widgets.Text(
    value=gServerUrl,
    placeholder='Type something',
    description='PiE Server URL:',
    disabled=False,
    style=urlStyle
)

#
# button to reload data from REST
reloadButtonWidget = widgets.Button(
    description='Reload From Server',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='fa-refresh' #'check'
)

def reload_callback(b):
    print('reload_callback()')
    loaddata()
    
reloadButtonWidget.on_click(reload_callback)

#
# slider to control y-axis of lpot
#yAxisStyle = {'description_width': '200px', 'width': '600px'}
yAxisWidget = widgets.IntRangeSlider(
    value=[gMinY, gMaxY],
    min=gMinY,
    max=gMaxY,
    step=1,
    description='Y-axis Min/Max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    #style=yAxisStyle
)

def yaxis_callback(v):
    newLowerLimit = v['new'][0]
    newUpperLimit = v['new'][1]
    #print('yaxis_callback() newLowerLimit:', newLowerLimit, 'newUpperLimit:', newUpperLimit)
    global gMinY
    gMinY = newLowerLimit
    global gMaxY
    gMaxY = newUpperLimit
    myplot()
          
yAxisWidget.observe(yaxis_callback, names='value')

In [28]:
#
# MAIN
#

myplot()

hbox0 = widgets.HBox([serverUrlWidget,reloadButtonWidget]) 
hbox1 = widgets.HBox([startDateWidget, endDateWidget, numberOfDaysWidget])
hbox2 = widgets.HBox([yAxisWidget])
vbox = widgets.VBox([hbox0, hbox1, hbox2])
display(vbox, f0)

widgets.interact(myrefresh)
#interactiveHandle = widgets.interactive(myplot)


VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oVGV4dCh2YWx1ZT11J2h0dHA6Ly8xOTIuMTY4LjEuMTk6NTAxMC9lbnZpcm9ubWVudGxvZycsIGRlc2NyaXB0aW9uPXUnUGlFIFNlcnZlciDigKY=


RmlndXJlV2lkZ2V0KHsKICAgICdkYXRhJzogW3snbWFya2VyJzogeydsaW5lJzogeyd3aWR0aCc6IDF9LCAnb3BhY2l0eSc6IDEsICdzaXplJzogNX0sCiAgICAgICAgICAgICAgJ21vZGUnOiDigKY=


interactive(children=(Output(),), _dom_classes=(u'widget-interact',))

<function __main__.myrefresh>